In [1]:
import tensorflow as tf
import io
from PIL import Image
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras import backend as K

import ipywidgets as widgets
from IPython.display import display, Javascript, HTML, clear_output, IFrame
from ipywidgets import interact, interactive, fixed, interact_manual, AppLayout, GridspecLayout

from src.utils import model_dicts, array2bytes, create_model, get_conv_layers
from src.guidedBacprop import GuidedBackprop, deprocess_image
from src.gradcam import GradCAM, overlay_gradCAM
from src.filtersVis import extract_filter, vis_filter

In [2]:
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style,
                          layout=widgets.Layout(height='auto', width='auto'))

model_ls = ["--Select"] + list(model_dicts.keys())
layer_ls = ["--Select"]
method_ls = ["--Select","Filter vis", "Feature map vis", "DeConvnet","Guided Backprop"]
class_ls = ["--Select"]

uploader = widgets.FileUpload()
show_but = create_expanded_button("Show image","success")
header = widgets.HTML('<font color="#1f77b4" face="sans-serif"><center><h1>Visualize and Explain Networks</h1><h3>All pre-trained models (Keras) on ImageNet</h3><hr style="height:2px;border-width:0;color:gray;background-color:gray"></center></font>',
                      layout=widgets.Layout(height='auto'))
models = widgets.Dropdown(options=model_ls, description="Model: ", layout={"width":"auto"},disabled=False)
layers = widgets.Dropdown(options=layer_ls, description="Layer: ", layout={"width":"auto"},disabled=False)
vis_but = create_expanded_button("Visualize","info")
methods = widgets.Dropdown(options=method_ls, description="Vis method: ", layout={"width":"auto"},disabled=False)
explain_but = create_expanded_button("Predict & Explain","warning")
classes = widgets.Dropdown(options=class_ls, description="Vis class: ", layout={"width":"auto"},disabled=False)
deep_dream_but = create_expanded_button("Generate","primary")
deco = widgets.Image(
    value=open("images/lapan.jpg","rb").read(),
    format="jpg",
    width="auto",
    height="auto",
    align="center-align"
)

In [8]:
# Layout
grid = GridspecLayout(15,30,height="600px")
grid[0,:4] = uploader
grid[0,5:9] = show_but
grid[0:3,10:] = header
grid[2,:9] = models
grid[3,:9] = widgets.HTML('<hr style="height:2px;border-width:0;color:gray;background-color:gray"><font color="gray" face="sans-serif"><center><h4>Filter and Feature Map Vis</h4></center>')
grid[4,:9] = layers
grid[5,:9] = methods
grid[7,:9] = vis_but
grid[8,:9] = widgets.HTML('<hr style="height:2px;border-width:0;color:gray;background-color:gray"><font color="gray" face="sans-serif"><center><h4>Explanation</h4></center>')
grid[9,:9] = explain_but
grid[11,:9] = classes
grid[12,:9] = widgets.HTML('<hr style="height:2px;border-width:0;color:gray;background-color:gray"><font color="gray" face="sans-serif"><center><h4>Deep Dream</h4></center>')
grid[13,:9] = deep_dream_but
grid[4:12,10:] = deco
grid[14,10:] = widgets.HTML('<font color="gray" face="sans-serif"><center><h4>Lang Co, Hue, Vietnam</h4></center>')
display(grid)

def on_change_model(change):
    if change["type"] == "change" and change["name"] == "value":
        chosen_model = change["new"]
        if chosen_model == "--Select":
            layer_ls = ["--Select"]
        else:
            model = create_model(chosen_model)
            layers.options = get_conv_layers(model)
        
    
def show_button(sender):
    for _, file_info in uploader.value.items():
        image = Image.open(io.BytesIO(file_info['content']))
        
    img = np.array(image)
    uploaded_img = widgets.Image(value=array2bytes(img))
    grid[4:10,10:] = uploaded_img
    
def vis_button(sender):
    clear_output()
    display(grid)
    model_name = models.value
    model = create_model(model_name)
    layer_name = layers.value
    assert methods.value in method_ls, "Method must be in {}".format(method_ls)
    if methods.value == "Filter vis":
        filters = extract_filter(model=model,layer_name=layer_name)
        print("Visualize filters {} of {}".format(layer_name, model_name))
        print("Filter shape: {}".format(filters.shape))
        print("Visualize 3 first channels of max 6 first filters:")
        vis_filter(filters)
    
    
show_but.on_click(show_button)
models.observe(on_change_model)
vis_but.on_click(vis_button)


GridspecLayout(children=(FileUpload(value={}, description='Upload', layout=Layout(grid_area='widget001')), But…